# Cutting Stock Column Generation

We will implement the master and pricing problems studied during the course.
The master problem selects the board configurations which are generated by the sub-problem.

#### Master problem:

\begin{align*}
\min & \sum_c x_c & \\
\text{subject to:} && \\
& \sum_c n_{cs} x_c \geq d_s & \forall s  \quad (\Pi^*_s) \\
& x_c \in \mathbb{N} & \forall c
\end{align*}

#### Pricing problem:

\begin{align*}
\min \quad & 1 - \sum_s \Pi^*_s n_s & \\
\text{subject to:} && \\
& \sum_s l_s n_s \leq L & \\
& n_s \in \mathbb{N} & \forall s
\end{align*}

The core procedure of the column generation can be described as follow:
1. Create the master problem with some initial columns to have a feasible problem;
2. Solve the linear relaxation of the master problem;
3. Update the sub-problem with the dual variables and solve it;
4. If the objective of the pricing problem is negative, add the corresponding configuration as a new column to the master problem;
5. If any column has been added to the master problem, go to 2, otherwise stop.

Finally, we will create a master problem with all the columns that has been generated by the column generation and we will solve the integer problem. We should normally obtain a good solution of the Cutting Stock problem. The objective value of the column generation is a lower bound that can be used to estimate the quality of the solution found.

In [46]:
L = 110
size = [20, 45, 50, 55, 75]
demand = [48, 35, 24, 10, 8]
nbShelves = len(size)
Shelves = range(nbShelves)

### Exercices

#### 1. Implement the pricing, and then the master porblems with the column generation logic.

In [93]:
import math
import gurobipy as gp


def subproblem(duals: list[float] = [0] * nbShelves):
    mdl = gp.Model(name='cutting_stock_subproblem')

    ## Add variables
    number_of_shelves_per_type_in_configuration = mdl.addVars(Shelves, vtype=gp.GRB.INTEGER, name='number_of_shelves_per_type_in_configuration')

    ## Constraints
    configuration_has_to_fit_in_shelve = mdl.addConstr(
        gp.quicksum(size[s] * number_of_shelves_per_type_in_configuration[s] for s in Shelves) <= L,
        name="configuration_has_to_fit_in_shelve"
    )

    ## Objective
    objective = mdl.setObjective(
        1 - gp.quicksum(duals[s] * number_of_shelves_per_type_in_configuration[s] for s in Shelves),  # TODO: this should be 1 - ..., because it's the coefficient of your master problem
        sense=gp.GRB.MINIMIZE
    )

    mdl.optimize()

    return [round(number_of_shelves_per_type_in_configuration[s].x) for s in Shelves], mdl.getObjective().getValue()


Test the subproblem function:

In [96]:
subproblem(duals=[1] * nbShelves)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.5.0 23F79)
Gurobi Compute Server Worker version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model fingerprint: 0x591b0d15
Variable types: 0 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+01, 8e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective -4.0000000
Presolve removed 1 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: -4 
No other solutions better than -4

Optimal solution found 

([5, 0, 0, 0, 0], -4.0)

#### 2. Implement the master and solve it with some initial columns (steps 1-2 of the procedure)

In [80]:
#initial configuration
configs = [[L // s if r == s else 0 for r in size] for s in size]

master = gp.Model(name='cutting_stock_master')

## Add variables
produce_x_configurations_of_type = master.addVars(range(len(configs)), vtype=gp.GRB.CONTINUOUS, name='produce_x_configurations_of_type')

## Add constraints
demand_constraints = master.addConstrs((gp.quicksum(configs[c][s] * produce_x_configurations_of_type[c] for c in range(len(configs))) >= demand[s] for s in Shelves), name="meet_demand_of")

## Objective
objective = master.setObjective(gp.quicksum(produce_x_configurations_of_type[c] for c in range(len(configs))), sense=gp.GRB.MINIMIZE)

print('Master obj={:.2f}'.format(master.getObjective().getValue()))


Master obj=0.00


#### 3. Implement the column generation (step 1 - 5)
You should use the previous question.

In [101]:
from pprint import pprint
import numpy as np

# 1. Build the master problem

#initial configuration
configs = [[L // s if r == s else 0 for r in size] for s in size]

master = gp.Model(name='cutting_stock_master')
iteration = 0
## Add variables
produce_x_configurations_of_type = master.addVars(range(len(configs)), vtype=gp.GRB.CONTINUOUS, lb=0, name='produce_x_configurations_of_type')

## Add constraints
demand_constraints = master.addConstrs(gp.quicksum(configs[c][s] * produce_x_configurations_of_type[c] for c in range(len(configs))) >= demand[s] for s in Shelves)

## Objective
objective = master.setObjective(gp.quicksum(produce_x_configurations_of_type[c] for c in range(len(configs))), sense=gp.GRB.MINIMIZE)

sub_objective = -np.inf

while sub_objective < 0:
    print("Enter the while loop")
    if iteration > 0:
        pprint(new_configuration)
        configs.append(new_configuration)
        produce_x_configurations_of_type[len(configs) - 1] = master.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name=f'produce_x_configurations_of_type[{len(configs) - 1}]')

        # There should be a nicer way to simply add the new terms rather than add the constraints again.
        master.remove(master.getConstrs())
        demand_constraints = master.addConstrs(gp.quicksum(configs[c][s] * produce_x_configurations_of_type[c] for c in range(len(configs))) >= demand[s] for s in Shelves)

        # for shelve_index, constraint in demand_constraints.items():
        #     pass
            # master.getRow(constraint).addTerms(new_configuration[shelve_index], produce_x_configurations_of_type[len(configs) - 1])

        master.setObjective(gp.quicksum(produce_x_configurations_of_type[c] for c in range(len(configs))), sense=gp.GRB.MINIMIZE)

    # 2. Solve the linear relaxation of the master problem
    # TODO: solve the linear relaxation of the master problem with the new columns
    master.optimize()

    print(f'Iter={iteration:5d}, Master obj={master.getObjective().getValue():.2f}, Columns: total={len(configs):6d}')
    print('-------------------------------------------------------------------')
    
    # 3. Solve the sub-problem with the dual variables
    # TODO: solve the sub-problem
    new_configuration, sub_objective = subproblem([d.Pi for d in demand_constraints.values()])
    
    
    # 4. If the objective is negative, add the the new column to the master problem else break
    # TODO: add the column of necessary

    iteration += 1
   

# Print the solution
print('-------------------------------------------------------------------')
print("Configurations available: ")
pprint(configs)
print("Master obj={:.2f}".format(master.getObjective().getValue()))

Enter the while loop
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.5.0 23F79)
Gurobi Compute Server Worker version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 5 columns and 5 nonzeros
Model fingerprint: 0x2cbd1e17
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 5e+01]
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.2100000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.210000000e+01
Iter=    0, Master obj=52.10, Columns: total=     5
-------

### Solve the Cutting Stock Problem
We use the column generated, to solve the integer problem. So, we must:
1. Redefine a integer problem with all the columns at the beginning;
2. Solve this problem

In [ ]:
mdl = gp.Model(name='cutting_stock')

